IMPLEMENTATION DE L'ALGORITHME UCB 1

In [37]:
import numpy as np

class UCB1 :
    """we create a new type of object called UCB1 with 
    Attributes:
    _ _ _ _ _ _  
    n_arms:int the number of arms"""
    def __init__(self, n_arms):
        self.n_arms = n_arms  # numbers of arms
        self.counts = np.zeros(n_arms) # for any arms we store  the number of time that it was played in this numpy table initialize by full zeros
        self.values = np.zeros(n_arms) # we also done that for store the average for all arms
        self.total_counts = 0 # for store the moment t step by step

    def select_arm(self):

        """This methode return the maximum value of the upper bound cumullative"""
        if self.total_counts < self.n_arms:
            #chaque bras est joué une fois: c'est le tir initial
            return self.total_counts
        else:
            UCB1_values = self.values + np.sqrt(2 * np.log(self.total_counts) / self.counts)
            return np.argmax(UCB1_values)    

    def update(self, chosen_arm, reward):
        """ this methode is used to update our algorithm it is an recursive methode
        intput:
                - chosen_arm : is the arm considered as chosen
                - reward is the reward ontained through the choise  of the  arm called chosen_arm  """

        self.counts[chosen_arm] += 1
        self.total_counts += 1
        n = self.counts[chosen_arm]
        value = self.values[chosen_arm]
        new_value = ((n-1)/n)*value + (1/n)*reward #  il s'agit de la mise a jour de la moyenne empirique
        self.values[chosen_arm] = new_value

    




Simulation de l'algorithme

Nous souhaitons maintenant implémenter l’algorithme UCB1 et générer des graphes de regret, en considérant des récompenses de type Bernoulli et Gaussienne. Pour cela, nous considérons le cas de K=2 bras, c’est-à-dire deux sources de récompenses aléatoires :

Le premier bras génère des observations issues d’une variable aléatoire 𝑋∼Bernoulli(1/2), Le second bras génère des observations(recompenses) issues d’une variable aléatoire Y∼N(0,1) , c’est-à-dire une loi normale standard.
pour cela on crée une nouvelle fonction qui prend en paramètre un bras et retourne une distribution aléatoire suivant qu'elle est Gaussienne ou bernoulli

In [43]:
import matplotlib.pyplot as plt


def distribution_reward(arm):
        if arm == 0:
            reward = np.random.normal(loc=0, scale=1.0) # pour la distibution normale centrée et reduite
        elif arm == 1:
            reward = np.random.binomial(1, 0.5) # une bernoulli est une binomial 1
        else:
            raise ValueError(f"Arm {arm} invalide. Il faut arm == 0 ou 1.")    
        return np.clip(reward, 0, 1) # pour forcer les distributions entre [0,1]


n_rounds = 1000
algo = UCB1(n_arms=2)
rewards = []
chosen_arms = []
# Espérances des bras (estimées théoriquement)
expected_rewards = [0.5, 0.5]  # pour normal(0,1) tronquée et Bernoulli(0.5), approx.
best_arm_mean = max(expected_rewards)
regret = []

cumulative_reward = 0

#simulation
for _ in range(n_rounds):
    arm = algo.select_arm()
    reward = distribution_reward(arm)
    algo.update(arm, reward)
    rewards.append(reward)
    chosen_arms.append(arm)
    cumulative_reward += reward


    # Regret cumulé à t
    r_t = _ * best_arm_mean - cumulative_reward
    regret.append(r_t)

# on montre les resultats

print("Recompense moyenne:", np.mean(rewards))
print("Bras sélectionnés:", algo.counts)



# Visualisation
plt.figure(figsize=(10,4))
plt.hist([a for i, a in enumerate(chosen_arms) if a == 0], bins=30, alpha=0.5, label="Bras 0 (Normal)")
plt.hist([a for i, a in enumerate(chosen_arms) if a == 1], bins=30, alpha=0.5, label="Bras 1 (bernoulli)")
plt.legend()
plt.title("Distribution des choix de bras")
plt.xlabel("Tour")
plt.ylabel("Nombre de fois choisi")
plt.show()

Recompense moyenne: 0.4841549939379822
Bras sélectionnés: [159. 841.]


In [44]:
# Affichage : courbe de regret
plt.figure(figsize=(8,5))
plt.plot(regret, label="Regret cumulé", color='red')
plt.xlabel("Tours")
plt.ylabel("Regret cumulé")
plt.title("Regret cumulé de l'algorithme UCB1")
plt.grid(True)
plt.legend()
plt.show()